In [14]:
import talib
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import datetime
import MetaTrader5 as mt5

In [19]:
class LiveTrade:
    
    def __init__(self):
        
#         self.df = pd.DataFrame(yf.download(tickers=instrument, start = '2022-10-01', end = '2023-06-01', interval='1h'))
        if not mt5.initialize(login=40987, server="CabanaCapitals-Demo",password="Mypassword$1234"):
            print("initialize() failed, error code =",mt5.last_error())
            quit()
        
        login = 40987
        password = 'Mypassword$1234'
        server = 'CabanaCapitals-Demo'
        mt5.login(login, password, server)
        
        rate = mt5.copy_rates_from('GBPUSD', mt5.TIMEFRAME_M15, datetime.datetime.now(), 1000)
        nf = pd.DataFrame(rate)
#         print('getting data')

        nf['date']=pd.to_datetime(nf['time'], unit='s')

        nf.set_index(np.arange(len(nf)), inplace = True)
        nf = nf.drop(['spread', 'real_volume', 'tick_volume', 'time'], axis='columns')


        self.df = nf.iloc[:500]  # First half
        self.df2 = nf.iloc[500:]  # Second half
        
        self.result = pd.DataFrame(columns=['ENTRY DATE', 'ENTRY', 'QUANTITY', 'TRADE', 'EXIT', 'EXIT DATE', 'P/L', 'REAL P/L', 'S/L' ])

        #     For counting 20 candles after climax

        self.buy_flag = True
        self.sell_flag = True
        self.green_count = 0
        self.red_count = 0
        
#         self.df['tradable'] = 0
        self.df['prev_candle'] = False
        self.df['candle_size'] = False
        
        
            
    def candle(self, df):
            '''Return candle colour'''

            if df['close'] - df['open'] > 0:
                return 'G'

            else:
                return 'R'

    def get_kc(self, high, low, close, kc_lookback, multiplier, atr_lookback):
        tr1 = pd.DataFrame(high - low)
        tr2 = pd.DataFrame(abs(high - close.shift()))
        tr3 = pd.DataFrame(abs(low - close.shift()))

        # Calculate True Range as the element-wise maximum of the three components
        tr = pd.concat([tr1, tr2, tr3], axis=1).max(axis=1)
        atr = tr.ewm(alpha=1/atr_lookback).mean()

        kc_middle = close.ewm(span=kc_lookback).mean()
        kc_upper = kc_middle + multiplier * atr
        kc_lower = kc_middle - multiplier * atr

        return kc_middle.iloc[-1], kc_upper.iloc[-1], kc_lower.iloc[-1]

    def calculate_indicators(self):
        
#         print('calc_indicator')
        length = len(self.df) - 1

        self.df.loc[length, 'EMA_8'] = talib.EMA(self.df['close'], timeperiod=8).iloc[-1]
        self.df.loc[length, 'EMA_50'] = talib.EMA(self.df['close'], timeperiod=50).iloc[-1]
        self.df.loc[length, 'EMA_200'] = talib.EMA(self.df['close'], timeperiod=200).iloc[-1]
        self.df.loc[length, 'RSI'] = talib.RSI(self.df['close'], timeperiod=21).iloc[-1]

        self.df.loc[length, 'candle'] = self.candle(self.df.iloc[-1])

#         1 ATR KT
#         (self.df.loc[length,'kc_middle'], self.df.loc[length,'kc_upper'],
#                     self.df.loc[length,'kc_lower']) = (self.get_kc(self.df['high'], 
#                                                     self.df['low'], self.df['close'],
#                                                     kc_lookback=20, multiplier=1, atr_lookback=10))

            
#        2.25 ATR KT
        
        self.df.loc[length, 'EMA_20'], self.df.loc[length, 'upper_keltner'],self.df.loc[length, 'lower_keltner'] = self.get_kc(self.df['high'], self.df['low'],self.df['close'], kc_lookback=20, multiplier=2.25, atr_lookback=10)
                                                        
                                                        
    def check_body(self, df):
        #     Calculate 0.7 % of entry price
    #     fOR 1H FOREX
            one_percent = df['open']/100 
            percent = one_percent * 0.06
            hammer_per = one_percent * 0.06

            print('Percent: ', percent)
        #     Calculate candle size close-open
            if df['candle'] == 'G':
                candle_size = df['close'] - df['open']
                upper_wick = df['high'] - df['close']
                lower_wick = df['open'] - df['low']

            else:
                candle_size = abs(df['close'] - df['open'])
                upper_wick = df['high'] - df['open']
                lower_wick = df['close'] - df['low']

            upper_wick_per = (upper_wick/one_percent) 
            lower_wick_per = (lower_wick/one_percent) 



        #     check for entry candle size and Max candle size 0.12 %
            if ((candle_size >= percent) and candle_size <= (0.12 * one_percent) and
             (candle_size > upper_wick) and (candle_size > lower_wick) and
            (upper_wick_per < 0.06) and (lower_wick_per < 0.06)):
                
                if df['candle'] == 'G' and df['high'] < df['upper_keltner'] :

                    return 1
                
                elif df['candle'] == 'R' and df['low'] > df['lower_keltner'] :

                    return 1
                
                else:
                    
                    return 0

        #     condition for hammmer
        #     lower wick is atleast 50 % of total size 
        #     Green Hammer 
        # need to add max wick of opposite dir
            elif ( (df['candle'] == 'G') and (df['open'] - df['low']) >= ((df['high'] - df['low']) * 0.5) and
                  (candle_size >= hammer_per) and (candle_size > upper_wick)and
                  candle_size <= (0.12 * one_percent) and ((upper_wick_per < 0.06))  
                 and df['high'] < df['upper_keltner']):

                return 1


    #     For Red Hammer
    # need to add max wick of opposite dir
            elif ( (df['candle'] == 'R') and (df['high'] - df['open']) >= (abs(df['high'] - df['low']) * 0.5) and 
                  (candle_size >= hammer_per) and (candle_size > lower_wick)and candle_size <= (0.12 * one_percent) and 
                  ((lower_wick_per < 0.06)) and df['low'] > df['lower_keltner'] ):

                return 1


            else:
                return 0

                                     
                                                      
    
    def check_inside_kt(self, df):
                                                      
#         Take only candles which are in inner KT[1 ATR] and not touching
#         outer KT[2.25 ATR]
        
        if ( ( df['high'] < df['upper_keltner'] and df['low'] > df['lower_keltner'])):
                                                      
            if (df['candle'] == 'G' and df['RSI'] <= 68 and df['close'] > df['EMA_50'] and 
             df['EMA_50'] > df['EMA_200'] ):
            
                return 1
                                                      
            elif (df['candle'] == 'R' and df['RSI'] >= 32 and df['close'] < df['EMA_50'] and 
               df['EMA_50'] < df['EMA_200'] ):
        
                return 1
        
        else:
            return 0
                                                      

        
                  
    def check_trade(self, df):
        
        if df['candle'] == 'G':
            
            return 'B'
        
        else:
            
            return 'S'
                                                      
                                                      
    def check_prev_candle(self, entry_candle):

        entry_index = self.df.loc[self.df.date ==  entry_candle.date].index[0]
        entry_candle_size = abs(entry_candle['open'] - entry_candle['close'])

#       For kind of double bottom or Morning Star
                                                      
        if entry_candle['candle'] == 'G':
            
#             Check last 10 candles for a red candle
            for i in range(entry_index - 1, max(-1, entry_index - 11), -1):
                
                current_candle = self.df.iloc[i]
                current_candle_size = abs(current_candle['open'] - current_candle['close'])
                
                # Check if it's a 'Red' candle that is at least 50% the size of the 'Green' candle
                if (current_candle['candle'] == 'R' and
                    current_candle['open'] <= entry_candle['close'] and
                    current_candle_size >= 0.5 * entry_candle_size and 
                    current_candle_size <= 1.2 * entry_candle_size):
                
#                   No candle in between should be greater than 60 % of red candle
                    for j in range(i + 1, entry_index):
                                                      
                        intermediate_candle = self.df.iloc[j]
                                                      
                        if (abs(intermediate_candle['high'] - intermediate_candle['low']) <= 0.6 *
                            current_candle_size):

                            continue
                                                      
                        elif j == entry_index - 1:
                            
                            return True
                
                        else:
                                
                            return False
                                                      
#       To check Double top or Evening star
                                                      
        elif entry_candle['candle'] == 'R':
            
#             Check last 10 candles for a red candle
            for i in range(entry_index - 1, max(-1, entry_index - 11), -1):
                
                current_candle = self.df.iloc[i]
                current_candle_size = abs(current_candle['open'] - current_candle['close'])
                
                # Check if it's a 'Red' candle that is at least 50% the size of the 'Green' candle
                if (current_candle['candle'] == 'G' and
                    current_candle['open'] >= entry_candle['close'] and
                    current_candle_size >= 0.5 * entry_candle_size and 
                    current_candle_size <= 1.2 * entry_candle_size):
                
#                   No candle in between should be greater than 60 % of red candle
                    for j in range(i + 1, entry_index):
                                                      
                        intermediate_candle = self.df.iloc[j]
                                                      
                        if (abs(intermediate_candle['high'] - intermediate_candle['low']) <= 0.6 *
                            current_candle_size):

                            continue
                                                      
                        elif j == entry_index - 1:
                            
                            return True
                
                        else:
                                
                            return False
                
    
        return False
                                                      
                                                      

    #   SL Method

    def set_sl(self, df):
        '''Set SL 1% above/below to entry candle'''
            
        candle_size = df['high'] - df['low']

        if df['candle'] == 'G':

            sl = df['low'] - (candle_size/100)

        elif df['candle'] == 'R':

            sl = df['high'] + (candle_size/100)
        
#         print('SL: ', sl)

        return sl
        

    #   Apply all the conditions on current candle

    def check_conditions(self):
        
        length = len(self.df) - 1
        
#         Checking all entry conditions
        
        self.df.loc[length, 'pos'] = self.check_inside_kt(self.df.iloc[-1])
        
        self.df.loc[length, 'candle_size'] = self.check_body(self.df.iloc[-1])
                                                      
        self.df.loc[length, 'trade'] = self.check_trade(self.df.iloc[-1])
                                                      
        self.df.loc[length, 'prev_candle'] = self.check_prev_candle(self.df.iloc[-1])
                                                      
     
        if ( (self.df.loc[length, 'pos'] == 1) and (self.df.loc[length, 'candle_size'] == 1) and
           (self.df.loc[length, 'prev_candle'] == True) ):
            
#             print('Entry candle')
#             Set SL for ever entry
            self.df.loc[length, 'sl'] = self.set_sl(self.df.iloc[-1])
            
            self.result_len = len(self.result)

            self.result.loc[self.result_len, 'ENTRY DATE'] = self.df.loc[length, 'date']
            self.result.loc[self.result_len, 'ENTRY'] = self.df.loc[length, 'close']
            self.result.loc[self.result_len, 'TRADE'] = self.df.loc[length, 'trade']
            self.result.loc[self.result_len, 'S/L'] = self.df.loc[length, 'sl']
                                                      
                                                      
                                                      
    def Caller(self):
        # Initialize an empty DataFrame to store the past data that gets dropped
        self.past_df = pd.DataFrame()
        
#         Loop for taking the live data
        for index, row in self.df2.iterrows():
            # Add live data to the df DataFrame
            self.df = self.df.append(row, ignore_index=True)
#             self.df = pd.concat([self.df, row], ignore_index=True)

            # Delete the first row if the DataFrame has more than 202 rows
            if len(self.df) > 500:
                # Save the dropped row in past_df
                self.past_df = self.past_df.append(self.df.iloc[0], ignore_index=True)
                self.df = self.df.iloc[1:].reset_index(drop=True)
            

#             position_index = self.df[self.df['date'] == self.trend_date].index[0]
    
# #           Check trend
#             self.calculate_current_trend(position_index)
            
            # Calculate indicators for the current state of df
            self.calculate_indicators()
            self.check_conditions()
        
        # Concatenate past_df and df to get the full dataset
        self.result_df = pd.concat([self.past_df, self.df], ignore_index=True)

        # Save the result_df to an Excel file
        self.result_df.to_excel('new_combined.xlsx')

#         self.result.to_excel('new_Record_trades.xlsx')



In [20]:
ob = LiveTrade()

ob.Caller()

C:\Users\hp\AppData\Local\Temp\ipykernel_15008\2162255045.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.df['prev_candle'] = False
C:\Users\hp\AppData\Local\Temp\ipykernel_15008\2162255045.py:330: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.df = self.df.append(row, ignore_index=True)
C:\Users\hp\AppData\Local\Temp\ipykernel_15008\2162255045.py:336: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.past_df = self.past_df.append(self.df.iloc[0], ignore_index=True)
C:\Users\hp\AppData\Local\Temp\ipykernel_15008\2162255045.py:330: FutureWarning: The frame.a

Percent:  0.0007436999999999999
Percent:  0.000744882
Percent:  0.000744438
Percent:  0.000744246
Percent:  0.0007436640000000001
Percent:  0.0007439219999999999
Percent:  0.000744108
Percent:  0.0007436999999999999
Percent:  0.000743304
Percent:  0.000743802
Percent:  0.0007436699999999999
Percent:  0.000743958
Percent:  0.0007439519999999999
Percent:  0.0007437899999999999
Percent:  0.0007435319999999999
Percent:  0.0007436400000000001
Percent:  0.000743592
Percent:  0.000743328
Percent:  0.000743022
Percent:  0.00074325
Percent:  0.000742902
Percent:  0.0007427880000000001
Percent:  0.0007430819999999999
Percent:  0.0007430339999999998
Percent:  0.000742842
Percent:  0.000743112
Percent:  0.000743076
Percent:  0.000743124
Percent:  0.00074319
Percent:  0.00074325
Percent:  0.0007433819999999999
Percent:  0.000743454
Percent:  0.0007433819999999999
Percent:  0.000743562
Percent:  0.0007435319999999999
Percent:  0.0007436820000000001
Percent:  0.00074382
Percent:  0.000743856
Percent:

C:\Users\hp\AppData\Local\Temp\ipykernel_15008\2162255045.py:330: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.df = self.df.append(row, ignore_index=True)
C:\Users\hp\AppData\Local\Temp\ipykernel_15008\2162255045.py:336: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.past_df = self.past_df.append(self.df.iloc[0], ignore_index=True)


Percent:  0.000742602
Percent:  0.000742626
Percent:  0.0007427099999999999
Percent:  0.0007428599999999999
Percent:  0.0007428359999999999
Percent:  0.000743046
Percent:  0.000743628
Percent:  0.0007439460000000001
Percent:  0.000744048
Percent:  0.000743634
Percent:  0.0007437299999999998
Percent:  0.00074382
Percent:  0.000743694
Percent:  0.0007438139999999999
Percent:  0.000743598
Percent:  0.00074361
Percent:  0.00074319
Percent:  0.0007430399999999999
Percent:  0.0007428539999999999
Percent:  0.00074274
Percent:  0.0007424879999999999
Percent:  0.00074268
Percent:  0.000742866
Percent:  0.000742782
Percent:  0.000742908
Percent:  0.0007431839999999999
Percent:  0.00074316
Percent:  0.000743154
Percent:  0.000742908
Percent:  0.00074319
Percent:  0.000743208
Percent:  0.00074319
Percent:  0.00074319
Percent:  0.0007430819999999999
Percent:  0.000743016
Percent:  0.000742992
Percent:  0.00074301
Percent:  0.0007431
Percent:  0.00074298
Percent:  0.0007429559999999999
Percent:  0.0

C:\Users\hp\AppData\Local\Temp\ipykernel_15008\2162255045.py:330: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.df = self.df.append(row, ignore_index=True)
C:\Users\hp\AppData\Local\Temp\ipykernel_15008\2162255045.py:336: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.past_df = self.past_df.append(self.df.iloc[0], ignore_index=True)


Percent:  0.000739962
Percent:  0.00073992
Percent:  0.0007397459999999999
Percent:  0.00073959
Percent:  0.000739668
Percent:  0.0007392119999999998
Percent:  0.0007387859999999998
Percent:  0.0007387800000000001
Percent:  0.0007386239999999999
Percent:  0.0007389660000000001
Percent:  0.0007390979999999999
Percent:  0.0007391819999999999
Percent:  0.000739272
Percent:  0.000738876
Percent:  0.00073833
Percent:  0.000738546
Percent:  0.0007388879999999999
Percent:  0.0007391519999999999
Percent:  0.0007393140000000001
Percent:  0.000739302
Percent:  0.0007389480000000001
Percent:  0.000739248
Percent:  0.000739242
Percent:  0.0007391819999999999
Percent:  0.0007393739999999999
Percent:  0.0007391699999999999
Percent:  0.0007392
Percent:  0.0007391519999999999
Percent:  0.0007391699999999999
Percent:  0.000738834
Percent:  0.0007393140000000001
Percent:  0.000739608
Percent:  0.0007394760000000001
Percent:  0.0007389839999999999
Percent:  0.00073782
Percent:  0.0007387919999999999
Perc

C:\Users\hp\AppData\Local\Temp\ipykernel_15008\2162255045.py:330: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.df = self.df.append(row, ignore_index=True)
C:\Users\hp\AppData\Local\Temp\ipykernel_15008\2162255045.py:336: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.past_df = self.past_df.append(self.df.iloc[0], ignore_index=True)


Percent:  0.0007354919999999999
Percent:  0.000736524
Percent:  0.0007360859999999999
Percent:  0.0007356719999999998
Percent:  0.000735354
Percent:  0.000735264
Percent:  0.0007360379999999998
Percent:  0.000736536
Percent:  0.00073677
Percent:  0.0007363679999999999
Percent:  0.0007357259999999999
Percent:  0.000736578
Percent:  0.00073725
Percent:  0.000737322
Percent:  0.0007372139999999999
Percent:  0.0007379099999999999
Percent:  0.0007377059999999999
Percent:  0.0007374959999999999
Percent:  0.000737802
Percent:  0.00073788
Percent:  0.000737946
Percent:  0.0007378739999999999
Percent:  0.0007377779999999999
Percent:  0.000738006
Percent:  0.000738276
Percent:  0.000738108
Percent:  0.000737922
Percent:  0.000738102
Percent:  0.0007379880000000001
Percent:  0.00073782
Percent:  0.00073734
Percent:  0.000737736
Percent:  0.000737616
Percent:  0.0007377299999999999
Percent:  0.000737538
Percent:  0.000737454
Percent:  0.0007375319999999999
Percent:  0.000737616
Percent:  0.0007376

PermissionError: [Errno 13] Permission denied: 'new_combined.xlsx'